# Um agente que executa funções e notifica via telegram

### Setando nosso modelo

In [4]:
from agents import trace, Agent, Runner, function_tool, AsyncOpenAI, OpenAIChatCompletionsModel

In [5]:
client_groq = AsyncOpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

model_groq = OpenAIChatCompletionsModel(
    model="llama3-8b-8192",
    openai_client=client_groq
)

### Workflow do agente

In [10]:
instrucao1 = """
Você é um agente de vendas que trabalha na RamonSA,
uma empresa que oferece uma ferramenta SaaS para garantir a conformidade com o SOC2 e preparar auditorias, impulsionada por IA.
Você escreve mensagens frias profissionais e sérias para clientes que a receberão via telegram
"""

instrucao2 = """
Você é um agente de vendas bem-humorado e envolvente que trabalha na RamonSA,
uma empresa que oferece uma ferramenta SaaS para garantir a conformidade com o SOC2 e preparar auditorias, impulsionada por IA.
Você escreve mensagens frias, espirituosas e envolventes, com grande chance de obter uma resposta de clientes que a receberão via telegram
"""

instrucao3 = """
Você é um agente de vendas atarefado que trabalha na RamonSA,
uma empresa que oferece uma ferramenta SaaS para garantir a conformidade com o SOC2 e preparar auditorias, impulsionada por IA.
Você escreve mensagens frias, curtos e diretas ao ponto para clientes que a receberão via telegram
"""

In [12]:
sales_agent1 = Agent(
    name="Agente de vendas profissional",
    instructions=instrucao1,
    model=model_groq
)

sales_agent2 = Agent(
    name="Agente de vendas engajado",
    instructions=instrucao2,
    model=model_groq
)

sales_agent3 = Agent(
    name="Agente de vendas ocupado",
    instructions=instrucao3,
    model=model_groq
)

In [18]:
# Testando
result =  Runner.run_streamed(sales_agent1, input="Escreva uma mensagem para venda")

In [19]:
from openai.types.responses import ResponseTextDeltaEvent

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush="True")

"Alô [nome do cliente], espero que você esteja bem. Lembrei que nossa empresa RamonSA oferece uma solução SaaS revolucionária para garantir a conformidade com o SOC2 e preparar auditorias. Nossa ferramenta, impulsionada por IA, ajuda a identificar e mitigar riscos, simplificando o processo de certificação. Estamos aqui para ajudá-lo a melhorar a segurança e a confiança das suas operações. Gostaria de agendar um reunião para apresentar como podemos ajudá-lo? Responda e escolha um horário que melhor lhe atenda."

In [28]:
# Testando as coisas em paralelo
import asyncio

message = "Escreva uma mensagem de venda que irá para pessoas que a receberão via APP do telegram"

results = await asyncio.gather(
    Runner.run(starting_agent=sales_agent1, input=message),
    Runner.run(starting_agent=sales_agent2, input=message),
    Runner.run(starting_agent=sales_agent3, input=message),
)

In [29]:
all_results = [result.final_output for result in results]

for output in all_results:
    print(output + "\n\n")

"Olá [nome do cliente],

Espero que você esteja bem. Nossa equipe da RamonSA gostaria de apresentar sua empresa para a nossa ferramenta SaaS de conformidade com o SOC2. Como uma empresa que valoriza a segurança e a integridade dos dados, sabemos que a escolha certa de tecnologia é fundamental.

Nossa plataforma utiliza inteligência artificial para ajudar a automatizar e a simplificar o processo de conformidade, reduzindo significativamente o tempo e os custos associados às auditorias. Além disso, nossa ferramenta oferece uma visão real-time dos resultados e uma capacidade de monitoramento em tempo real, garantindo que sua empresa esteja sempre preparada para uma auditoria.

Gostaríamos de demonstrar como nossa ferramenta pode ajudar sua empresa a melhorar a eficiência e a reduzir os riscos de uma auditoria. Estamos dispostos a oferecer uma demonstração personalizada para você e sua equipe.

Por favor, respondendo a essa mensagem, podemos agendar um horário que melhor atenda às suas nec

### Definindo o agente que irá escolher a melhor mensagem dentre as três geradas acima

In [34]:
sales_picker = Agent(
    name="Agente que escolhe a melhor mensagem",
    instructions="""
    Você é o melhor em escolher a melhor mensagem para ser enviada à clientes que à receberão
    via APP do telegram. Devolva apenas a mensagem escolhida por você, sem mais e nem menos
""",
    model=model_groq
)

In [35]:
message = "Escreva uma mensagem de venda que irá para pessoas que a receberão via APP do telegram"

results = await asyncio.gather(
    Runner.run(starting_agent=sales_agent1, input=message),
    Runner.run(starting_agent=sales_agent2, input=message),
    Runner.run(starting_agent=sales_agent3, input=message),
)

all_results = [result.final_output for result in results]

all_messages = "Todas as mensagens:\n\n".join(all_results)
best_message = await Runner.run(starting_agent=sales_picker, input=all_messages)

In [36]:
print(f"A melhor mensagem foi: \n\n{best_message.final_output}")

A melhor mensagem foi: 

"Olá ! Fiquei sabendo que você busca uma solução para garantir a conformidade sua com o SOC2 e facilitar a preparação para auditorias. Eu estou trabalhando atualmente com a RamonSA, uma empresa líder em SaaS que utiliza inteligência artificial para impulsionar a gestão de compliance. Nossa ferramenta pode ajudar a reduzir o tempo e esforço necessários para atingir a conformidade, garantindo a segurança dos seus dados e melhorando a confiança com seus clientes. Estou disponível para discutir como a RamonSA pode ajudá-lo. Qual é o seu maior desafio atualmente em relação à conformidade do SOC2?"


### Fazendo a mesma coisa, mas usando uma tool

Aqui estamos transformando agentes em tools e um agente geral irá decidir qual agente utilizar.
Esse método é conhecido como hierárquico

In [37]:
import requests
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

chat_id = os.getenv("TELEGRAM_CHAT_ID")
token = os.getenv("TELEGRAM_TOKEN")

@function_tool
def send_message(msg):
    """
    Função que manda mensagem para o telegram

    ----------
    parameters
    ----------

    msg: str
        - parâmetro contendo a mensagem que será enviada
    """
    data = {"chat_id": chat_id, "text": msg}
    url = "https://api.telegram.org/bot{}/sendMessage".format(token)
    requests.post(url, data)
    return {"status": "ok"}

In [40]:
task = "Escreva uma mensagem de venda que irá para pessoas que a receberão via APP do telegram"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1",tool_description=task)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2",tool_description=task)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3",tool_description=task)

tools = [tool1, tool2, tool3, send_message]

In [44]:
general_instructions = """
Você é um gerente de vendas trabalhando para a empresa RamonSA.
Você irá usar as ferramentas dadas à você para gerar mensagens para clientes no app do telegram.
Você nunca deverá gerar mensagens sozinho, mas sim usar as ferramentas sempre que for realizar tal tarefa.
Você irá tentar todas as 3 ferramentas de sales_agent e escolher a melhor dentre elas para ser mandada.
E irá mandar a mensagem usando a função send_message
"""

sales_manager = Agent(
    name="Gerente de vendas",
    instructions=general_instructions,
    tools=tools,
    model=model_groq
)

In [45]:
message = "Escreva uma mensagem de venda que irá para pessoas que a receberão via APP do telegram"


with trace("Sales manager"):
    result_manager = await Runner.run(
        starting_agent=sales_manager,
        input=message
    )